# __ASSIGNMENT 5__

## __IMPORT LIBRARIES__

In [18]:
import pandas as pd
import numpy as np


## __1.__

In [19]:
data = pd.read_csv('height_weight_genders.csv')
data.head()

,Gender,Height,Weight
0,Male,73.847017,241.893563
1,Male,68.781904,162.310473
2,Male,74.110105,212.740856
3,Male,71.730978,220.042470
4,Male,69.881796,206.349801


In [20]:
def replace_gender(g):
    if g == "Male": 
        return 0
    else:
        return 1

data["Gender"] = data["Gender"].apply(replace_gender) 
data.head()
        

,Gender,Height,Weight
0,0,73.847017,241.893563
1,0,68.781904,162.310473
2,0,74.110105,212.740856
3,0,71.730978,220.042470
4,0,69.881796,206.349801


In [21]:
data = np.array([data["Gender"],data["Height"], data["Weight"]]).T

In [22]:
print(data.shape)

(10000, 3)


## __2.__

In [ ]:
# NOTE: use vectorized operationsssss !!!!!!
h = data[:, 1]

mn = np.mean(h)
std_dev = np.std(h)

data[:, 1] = (data[:, 1] - mn) / std_dev

print(data[:, 1])

[ 1.94406149  0.62753668  2.01244346 ... -0.64968792  0.69312469
 -1.14970831]


## __3.__

In [32]:
Z = np.array([np.ones((data.shape[0],1)), data[:, 1].reshape(-1,1)]).T
Y = data[:, 2].reshape(-1,1)
print(Z.shape)
print(Y.shape)

(1, 10000, 2)
(10000, 1)


## __4.__

In [ ]:
# NOTE: MEAN SQUARED ERROR 
def objective(y,Z,x):
    return (1/(2*y.shape[0])) * np.sum((y - Z @ x)**2)

## __5.__

In [34]:
def gradient(y,Z,x):
    return 1/y.shape[0] * Z.T (Z*(x) - y) 

## __6.__